In [1]:
import pandas as pd


In [2]:
transactions_df = pd.read_csv("transactions_tm1_e.csv")
customers_df = pd.read_csv("customers_tm1_e.csv")

In [3]:
customers_df.head()

,customer_id,dob,state,start_balance,creation_date
0,91,1993-07-01,California,10180.56,2007-01-31
1,92,1985-12-05,New York,4757.68,2007-01-31
2,93,1987-11-19,Minnesota,6796.72,2007-01-31
3,94,1981-03-23,Minnesota,9870.48,2007-01-31
4,95,1970-04-06,California,12500.72,2007-01-31


## Same number of customer_id, account_id combos as there are account_ids and customer_ids (they are the same )

In [4]:
len(customers_df)

116016

In [5]:
transactions_df.head()

,date,account_id,customer_id,amount,transaction_date,deposit,withdrawal
0,2007-01-31,24137947,91,3034.26,2007-01-31,3034.26,0.00
1,2007-01-31,24137947,91,-5295.18,2007-01-16,0.00,-5295.18
2,2007-02-28,24137947,91,0.00,2007-02-28,0.00,0.00
3,2007-03-31,24137947,91,-0.00,2007-03-30,0.00,-0.00
4,2007-03-31,24137947,91,-0.00,2007-03-11,0.00,-0.00


In [6]:
account_ids = transactions_df.account_id.value_counts()

In [7]:
#number of account ids that have made transactions 
print(len(account_ids))

116016


In [8]:
df2 = transactions_df.groupby(['customer_id','account_id']).agg({'customer_id': ['count'],'account_id': ['count']})
len(df2)

116016

## Agregate by Month

In [9]:
months = []
for n in transactions_df["date"]:
    months.append(n[:7])
transactions_df["month/year"] = months

In [10]:
transactions_df.head()

,date,account_id,customer_id,amount,transaction_date,deposit,withdrawal,month/year
0,2007-01-31,24137947,91,3034.26,2007-01-31,3034.26,0.00,2007-01
1,2007-01-31,24137947,91,-5295.18,2007-01-16,0.00,-5295.18,2007-01
2,2007-02-28,24137947,91,0.00,2007-02-28,0.00,0.00,2007-02
3,2007-03-31,24137947,91,-0.00,2007-03-30,0.00,-0.00,2007-03
4,2007-03-31,24137947,91,-0.00,2007-03-11,0.00,-0.00,2007-03


In [13]:
grp_df = transactions_df.groupby(['account_id', 'month/year', 'customer_id']).agg({'amount': ['sum'], 
                                                                                   'customer_id' : ['count']})
grp_df.columns = ['this_month_transactions', 'this_month_num_transactions']
grp_df = grp_df.reset_index()

## aggregated by month 

In [14]:
grp_df.head()

,account_id,month/year,customer_id,this_month_transactions,this_month_num_transactions
0,24137947,2007-01,91,-2260.92,2
1,24137947,2007-02,91,0.00,1
2,24137947,2007-03,91,0.00,3
3,24137948,2007-01,92,0.00,1
4,24137948,2007-02,92,1164.90,1


## Timeframe of data

In [15]:
print(grp_df['month/year'].min())
print(grp_df['month/year'].max())
print(len(grp_df))

2007-01
2020-05
2129121


## adding interest rates to transactions data 

In [16]:
interest_rates = pd.read_csv("FEDFUNDS.csv")
interest_rates.head()

,DATE,FEDFUNDS
0,1954-07-01,0.80
1,1954-08-01,1.22
2,1954-09-01,1.07
3,1954-10-01,0.85
4,1954-11-01,0.83


In [17]:
type(interest_rates['DATE'][0])

str

In [18]:
months = []
for n in interest_rates["DATE"]:
    months.append(n[:7])
interest_rates["month/year"] = months

In [19]:
interest_rates = interest_rates[["FEDFUNDS", "month/year"]]

In [20]:
interest_rates.head()

,FEDFUNDS,month/year
0,0.80,1954-07
1,1.22,1954-08
2,1.07,1954-09
3,0.85,1954-10
4,0.83,1954-11


In [21]:
grp_df = grp_df.join(interest_rates.set_index('month/year'), on='month/year', how='left', lsuffix='_tran', rsuffix='_fed')

## Added month's interest rate into table

In [22]:
grp_df.head()

,account_id,month/year,customer_id,this_month_transactions,this_month_num_transactions,FEDFUNDS
0,24137947,2007-01,91,-2260.92,2,5.25
1,24137947,2007-02,91,0.00,1,5.26
2,24137947,2007-03,91,0.00,3,5.26
3,24137948,2007-01,92,0.00,1,5.25
4,24137948,2007-02,92,1164.90,1,5.26


## Lag transaction amounts 
so that we can use previous 2 months of transactions to predict whether a customer will churn in the current month 

In [23]:
grp_df['last_month_transactions'] = grp_df.groupby(['customer_id'])['this_month_transactions'].shift(1)

In [24]:
grp_df['2_months_ago_transactions'] = grp_df.groupby(['customer_id'])['this_month_transactions'].shift(1)

In [25]:
grp_df = pd.merge(grp_df, customers_df, how='inner', on='customer_id')

In [26]:
grp_df.head()

,account_id,month/year,customer_id,this_month_transactions,this_month_num_transactions,FEDFUNDS,last_month_transactions,2_months_ago_transactions,dob,state,start_balance,creation_date
0,24137947,2007-01,91,-2260.92,2,5.25,NaN,NaN,1993-07-01,California,10180.56,2007-01-31
1,24137947,2007-02,91,0.00,1,5.26,-2260.92,-2260.92,1993-07-01,California,10180.56,2007-01-31
2,24137947,2007-03,91,0.00,3,5.26,0.00,0.00,1993-07-01,California,10180.56,2007-01-31
3,24137948,2007-01,92,0.00,1,5.25,NaN,NaN,1985-12-05,New York,4757.68,2007-01-31
4,24137948,2007-02,92,1164.90,1,5.26,0.00,0.00,1985-12-05,New York,4757.68,2007-01-31


## Adding Account balances from customer table to transactions 
Also keeps track of them month by month based on the transactions amount 

In [27]:
balances = {}
for i, n in customers_df.iterrows():
    balances[n["customer_id"]] = n["start_balance"]

In [28]:
balance_rows = []
for i, n in grp_df.iterrows():
    customer_id = n["customer_id"]
    previous_balance = balances[customer_id]
    new_balance = previous_balance+n["this_month_transactions"]
    balances[customer_id] = new_balance
    balance_rows.append(new_balance)
grp_df["this_month_balance"] = balance_rows

In [29]:
grp_df.head()

,account_id,month/year,customer_id,this_month_transactions,this_month_num_transactions,FEDFUNDS,last_month_transactions,2_months_ago_transactions,dob,state,start_balance,creation_date,this_month_balance
0,24137947,2007-01,91,-2260.92,2,5.25,NaN,NaN,1993-07-01,California,10180.56,2007-01-31,7919.64
1,24137947,2007-02,91,0.00,1,5.26,-2260.92,-2260.92,1993-07-01,California,10180.56,2007-01-31,7919.64
2,24137947,2007-03,91,0.00,3,5.26,0.00,0.00,1993-07-01,California,10180.56,2007-01-31,7919.64
3,24137948,2007-01,92,0.00,1,5.25,NaN,NaN,1985-12-05,New York,4757.68,2007-01-31,4757.68
4,24137948,2007-02,92,1164.90,1,5.26,0.00,0.00,1985-12-05,New York,4757.68,2007-01-31,5922.58


## Lag balances  
so that we can use previous 2 months of balances to predict whether a customer will churn in the current month 

In [30]:
grp_df['last_month_balance'] = grp_df.groupby(['customer_id'])['this_month_balance'].shift(1)

In [31]:
grp_df['2_months_ago_balance'] = grp_df.groupby(['customer_id'])['this_month_balance'].shift(2)

In [32]:
grp_df.head()

,account_id,month/year,customer_id,this_month_transactions,this_month_num_transactions,FEDFUNDS,last_month_transactions,2_months_ago_transactions,dob,state,start_balance,creation_date,this_month_balance,last_month_balance,2_months_ago_balance
0,24137947,2007-01,91,-2260.92,2,5.25,NaN,NaN,1993-07-01,California,10180.56,2007-01-31,7919.64,NaN,NaN
1,24137947,2007-02,91,0.00,1,5.26,-2260.92,-2260.92,1993-07-01,California,10180.56,2007-01-31,7919.64,7919.64,NaN
2,24137947,2007-03,91,0.00,3,5.26,0.00,0.00,1993-07-01,California,10180.56,2007-01-31,7919.64,7919.64,7919.64
3,24137948,2007-01,92,0.00,1,5.25,NaN,NaN,1985-12-05,New York,4757.68,2007-01-31,4757.68,NaN,NaN
4,24137948,2007-02,92,1164.90,1,5.26,0.00,0.00,1985-12-05,New York,4757.68,2007-01-31,5922.58,4757.68,NaN


## dealing with Null values 

In [33]:
grp_df.isna().head()

,account_id,month/year,customer_id,this_month_transactions,this_month_num_transactions,FEDFUNDS,last_month_transactions,2_months_ago_transactions,dob,state,start_balance,creation_date,this_month_balance,last_month_balance,2_months_ago_balance
0,False,False,False,False,False,False,True,True,False,False,False,False,False,True,True
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,True,True,False,False,False,False,False,True,True
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [34]:
df = grp_df[grp_df['start_balance'].isna()]

In [35]:
df[df["state"] != "Missouri"]

,account_id,month/year,customer_id,this_month_transactions,this_month_num_transactions,FEDFUNDS,last_month_transactions,2_months_ago_transactions,dob,state,start_balance,creation_date,this_month_balance,last_month_balance,2_months_ago_balance
869400,24183838,2012-08,45982,3620.30,1,0.13,NaN,NaN,1991-09-15,New York,NaN,2012-08-31,NaN,NaN,NaN
869401,24183838,2012-09,45982,2485.50,1,0.14,3620.30,3620.30,1991-09-15,New York,NaN,2012-08-31,NaN,NaN,NaN
869402,24183838,2012-10,45982,-2724.01,3,0.16,2485.50,2485.50,1991-09-15,New York,NaN,2012-08-31,NaN,NaN,NaN
869403,24183838,2012-11,45982,-1704.74,3,0.16,-2724.01,-2724.01,1991-09-15,New York,NaN,2012-08-31,NaN,NaN,NaN
869404,24183838,2012-12,45982,0.00,4,0.16,-1704.74,-1704.74,1991-09-15,New York,NaN,2012-08-31,NaN,NaN,NaN


In [36]:
nulls = grp_df[grp_df.isna().any(axis=1)]

In [37]:
nulls.head()

,account_id,month/year,customer_id,this_month_transactions,this_month_num_transactions,FEDFUNDS,last_month_transactions,2_months_ago_transactions,dob,state,start_balance,creation_date,this_month_balance,last_month_balance,2_months_ago_balance
0,24137947,2007-01,91,-2260.92,2,5.25,NaN,NaN,1993-07-01,California,10180.56,2007-01-31,7919.64,NaN,NaN
1,24137947,2007-02,91,0.00,1,5.26,-2260.92,-2260.92,1993-07-01,California,10180.56,2007-01-31,7919.64,7919.64,NaN
3,24137948,2007-01,92,0.00,1,5.25,NaN,NaN,1985-12-05,New York,4757.68,2007-01-31,4757.68,NaN,NaN
4,24137948,2007-02,92,1164.90,1,5.26,0.00,0.00,1985-12-05,New York,4757.68,2007-01-31,5922.58,4757.68,NaN
18,24137949,2007-01,93,1809.75,1,5.25,NaN,NaN,1987-11-19,Minnesota,6796.72,2007-01-31,8606.47,NaN,NaN


In [38]:
len(nulls)

226675